In [1]:
import lightgbm as lgb
import xgboost as xgb 
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV
from kneed import KneeLocator
from sklearn.cluster import KMeans

import pandas as pd

In [2]:
train = pd.read_csv('train.csv')
x_train = train.drop("total_amount",axis=1)

In [29]:
wcss = []

for i in range(1,5):
    kmean = KMeans(i,random_state=47)
    kmean.fit(x_train)
    wcss.append(kmean.inertia_)

In [5]:
d = pd.DataFrame()

In [9]:
len(d)

0

In [11]:
if not len(d):
    print('yes')

yes


In [4]:
if train.shape:
    print('yes')

yes


In [30]:
kn = KneeLocator(range(1,5),wcss,curve='convex',
                        direction='decreasing')

In [31]:
kn.knee

2

In [20]:
# kmean = KMeans(kn.knee,random_state=47)
kmean = KMeans(100,random_state=47)

In [21]:
kmean.fit(x_train)

KMeans(n_clusters=100, random_state=47)

In [22]:
labels = kmean.labels_

In [23]:
labels

array([56, 40, 10, ..., 64, 70, 58])

In [24]:
da = pd.DataFrame(labels,columns=['labels'])

In [36]:
da['labels'].value_counts(ascending=True)

81        1
17        3
68        4
88        4
25        4
      ...  
32    33774
71    34266
56    34370
63    35176
21    35775
Name: labels, Length: 100, dtype: int64

In [40]:
df = pd.concat([train,da],axis=1)

In [41]:
df.shape

(838441, 10)

In [42]:
train.shape

(838441, 9)

In [43]:
new_0 = df[df["labels"]==0]

In [44]:
new_1 = df[df['labels']==1]

In [45]:
new_1

,driver_tip,distance,num_passengers,toll_amount,payment_method,rate_code,extra_charges,total_amount,trip_duration,labels
2292,0.00,0.92,1.0,0.0,2.0,1.0,0.0,9.30,52900.0,1
2810,3.36,4.05,1.0,0.0,1.0,1.0,0.0,20.16,86241.0,1
2910,4.20,4.15,1.0,0.0,1.0,1.0,1.0,21.00,85114.0,1
3814,1.66,0.95,1.0,0.0,1.0,1.0,0.0,9.96,84515.0,1
4830,3.00,2.26,1.0,0.0,1.0,1.0,0.0,19.30,84276.0,1
...,...,...,...,...,...,...,...,...,...,...
833148,0.00,2.94,1.0,0.0,1.0,1.0,0.5,13.80,86031.0,1
835062,0.00,2.92,1.0,0.0,2.0,1.0,0.0,16.80,86315.0,1
836163,0.00,0.57,1.0,0.0,2.0,1.0,1.0,8.30,85969.0,1
836283,0.00,0.82,2.0,0.0,2.0,1.0,0.0,8.80,83281.0,1


In [46]:
x_0 = new_0.drop(["total_amount",'labels'],axis=1)
y_0 = new_0['total_amount']

In [47]:
y_0

0          9.96
1         17.85
2         12.36
3         63.56
4          7.55
          ...  
838436    52.27
838437    15.80
838438     5.83
838439    15.95
838440    15.80
Name: total_amount, Length: 837348, dtype: float64

In [38]:
params = {"max_depth":8, "n_estimators": 750, "learning_rate": 0.025,"random_state":42}
model = xgb.XGBRegressor(**params)
model

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, gamma=None,
             gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.025, max_bin=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=8,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=750, n_jobs=None,
             num_parallel_tree=None, predictor=None, random_state=42,
             reg_alpha=None, reg_lambda=None, ...)

In [78]:
model = xgb.XGBRegressor(max_depth=8, n_estimators = 750, learning_rate= 0.025,random_state=42)
model.fit(x_0,y_0)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.025, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=8, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=750, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=42,
             reg_alpha=0, reg_lambda=1, ...)

In [80]:
test1 = pd.read_csv("valid.csv")
test = test.drop("total_amount",axis=1)
test_labels = kmean.predict(test)

In [81]:
da = pd.DataFrame(test_labels,columns=['label'])

In [82]:
testco = pd.concat([test,da],axis=1)

In [83]:
testco

,driver_tip,distance,num_passengers,toll_amount,payment_method,rate_code,extra_charges,trip_duration,label
0,0.00,1.27,4.0,0.0,2.0,1.0,0.0,466.0,0
1,1.92,2.60,1.0,0.0,1.0,1.0,0.0,832.0,0
2,0.00,0.50,1.0,0.0,2.0,1.0,0.5,91.0,0
3,1.15,0.50,1.0,0.0,1.0,1.0,1.0,157.0,0
4,5.00,3.50,1.0,0.0,1.0,1.0,0.0,1002.0,0
...,...,...,...,...,...,...,...,...,...
209606,0.00,0.71,1.0,0.0,2.0,1.0,0.5,160.0,0
209607,1.00,0.70,1.0,0.0,1.0,1.0,0.0,356.0,0
209608,2.85,0.90,1.0,0.0,1.0,1.0,0.0,1308.0,0
209609,0.00,4.06,1.0,0.0,1.0,1.0,0.5,1047.0,0


In [84]:
xtest0 = testco[testco['label']==0]

In [85]:
x_pred = xtest0.drop('label',axis=1)

In [88]:
x_pred.shape,test1.shape

((209333, 8), (209611, 9))

In [96]:
x_pred_test = test1.loc[x_pred.index,:]
x_pred_test.shape

(209333, 9)

In [97]:
y_true = x_pred_test['total_amount']

In [98]:
y_pred = model.predict(x_pred_test.drop(['total_amount'],axis=1))

In [103]:
len(y_true)

209333

In [102]:
len(y_pred)

209333

In [104]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_true,y_pred)

0.40701752239436184

In [76]:
model1 = GradientBoostingRegressor(n_estimators=10,
    max_leaf_nodes= 20,
    learning_rate=0.2,
    max_depth=4)

In [77]:
model1.fit(x_0,y_0)

GradientBoostingRegressor(learning_rate=0.2, max_depth=4, max_leaf_nodes=20,
                          n_estimators=10)

In [105]:
y_pred = model1.predict(x_pred_test.drop(['total_amount'],axis=1))

In [106]:
mean_absolute_error(y_true,y_pred)

1.5028269013025404

In [74]:
model2 = lgb.LGBMRegressor(task="train",
    boosting_type= "gbdt",
    objective= "regression",
    learning_rate= 0.005,
    feature_fraction= 0.9,
    bagging_fraction= 0.7,
    bagging_freq=10,
    verbose= 0,
    max_depth= 8,
    num_leaves= 128,  
    max_bin= 512,
    num_iterations= 100)

In [75]:
model2.fit(x_0,y_0)

C:\Users\sande\anaconda3\envs\mach\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.037102 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.


LGBMRegressor(bagging_fraction=0.7, bagging_freq=10, feature_fraction=0.9,
              learning_rate=0.005, max_bin=512, max_depth=8, num_iterations=100,
              num_leaves=128, objective='regression', task='train', verbose=0)

In [107]:
y_pred = model2.predict(x_pred_test.drop(['total_amount'],axis=1))

In [108]:
mean_absolute_error(y_true,y_pred)

5.423408347286484